In [33]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px


from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# HTML LAYOUT
app.layout = html.Div([
    html.Div(id='hidden-div', style={
        'max-width': 'fit-content',
        'margin-inline': 'auto'
    }, children=[
       html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())) 
    ]),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
#   Unique Identifier
    html.Center(html.H2('4/20/24')),
    html.Hr(),
    html.Div(
        
#interactive radio filters based on rescue type
       dcc.RadioItems(['Water Rescue', 'Wilderness Rescue','Disaster Rescue', 'Reset'], 'Reset', inline=True, id='filter-type') 
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#features for your interactive data table
        editable=False,
        sort_action="native",
        sort_mode="multi",
        filter_action='native',
        cell_selectable = False,
        row_selectable="single",
        row_deletable=False,
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
                        ),
    html.Br(),
    html.Hr(),
#Chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# Updates data table based on filter type    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
#     Query based on filter type
    filter_conditions = {
        'Reset': {},
        'Water Rescue': {
            "animal_type":"Dog",
            "breed":{
                "$in":[
                    "Labrador Retriever Mix",
                    "Chesapeake Bay Retriever",
                    "Newfoundland"
                ]
            }, 
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}
        }, 
        'Wilderness Rescue': {
            "animal_type":"Dog",
            "breed":{
                "$in":[
                    "German Shepherd", 
                    "Alaskan Malamute", 
                    "Old English Sheepdog", 
                    "Siberian Husky",
                    "Rottweiler"
                ]
            }, 
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}
        },
        'Disaster Rescue': {
            "animal_type":"Dog",
            "breed":{
                "$in":[
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound", 
                    "Rottweiler"
                ]
            }, 
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20},
            "age_upon_outcome_in_weeks":{"$lte":300}
        }
    }
    query = filter_conditions.get(filter_type, {})
    
    df = pd.DataFrame.from_records(db.read(query))
    
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

# Pie chart that displays the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # imports the currently displayed data
    dff = pd.DataFrame.from_dict(viewData)
    # Creates the values needed for the names (breed) and values (recurring counts)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    # Creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):
    if viewData is None:
        # No data available, return an empty map
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ])
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if selected_rows is None or len(selected_rows) == 0:
        row = 0
    else: 
        row = selected_rows[0]

    lon = dff.iloc[row]['location_long']
    lat = dff.iloc[row]['location_lat']
    breed = dff.iloc[row]['breed']
    name = dff.iloc[row]['name']
    
    
    map_children = [ 
        dl.TileLayer(id="base-layer-id"),
#               Marker with tool tip and popup
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([
                            html.H5("Animal Name"),
                            html.P(name)
                        ])
                    ])
    ]
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48], 
            zoom=10, 
            children= map_children
        )
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:30140/
Lon: -97.7419963476444
Lat: 30.6525984560228
Breed: Domestic Shorthair Mix
Name: 
Map Children: [TileLayer(id='base-layer-id'), Marker(children=[Tooltip('Domestic Shorthair Mix'), Popup([H5('Animal Name'), P('')])], position=[30.6525984560228, -97.7419963476444])]
Lon: -97.3087780473978
Lat: 30.4551148649096
Breed: Boxer/Bullmastiff
Name: *Kawhi
Map Children: [TileLayer(id='base-layer-id'), Marker(children=[Tooltip('Boxer/Bullmastiff'), Popup([H5('Animal Name'), P('*Kawhi')])], position=[30.4551148649096, -97.3087780473978])]
Lon: -97.5923916912722
Lat: 30.2709983761287
Breed: Bloodhound
Name: Boomer
Map Children: [TileLayer(id='base-layer-id'), Marker(children=[Tooltip('Bloodhound'), Popup([H5('Animal Name'), P('Boomer')])], position=[30.2709983761287, -97.5923916912722])]
Lon: -97.5492968638502
Lat: 30.329873203611
Breed: Rottweiler
Name: Striker
Map Children: [TileLayer(id='base-layer-id'), Marker(children=[Tooltip('Rottweiler'), Popup([H5